In [ ]:
import json
import os
import pandas as pd 
from pathlib import Path

image_path=r"/home/william/dataset/skin/SkinCAP/skincap"

file_path=r"/home/william/dataset/skin/SkinCAP/skincap_v240623.csv"
json_name ="SkinCAP.json"
dt=pd.read_csv(file_path)
SkinCAP_json = []
for index, data in dt.iterrows():
    disease = data['disease']
    caption_zh_polish_en = data['caption_zh_polish_en']
    skincap_file_path = data['skincap_file_path']
    caption_zh_polish = data['caption_zh_polish']
    caption_zh = data['caption_zh']
    if(disease.replace("-"," ") in caption_zh_polish_en):
        data_json={
            "image_name": skincap_file_path,
            "disease": disease,
            "caption_zh": caption_zh,
            "caption_zh_polish": caption_zh_polish,
            "caption_zh_polish_en":caption_zh_polish_en,
            
        }
        SkinCAP_json.append(data_json)
        

output_path = Path(json_name)
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(SkinCAP_json, f, ensure_ascii=False, indent=4)

print(f"JSON save: {output_path.resolve()}")    


# open_end_QA

In [ ]:
import json
import os
import pandas as pd 
from pathlib import Path
from datetime import datetime 

image_path=r"/home/william/dataset/skin/SkinCAP"

file_path=r"/home/william/dataset/skin/SkinCAP/skincap_v240623.csv"
today_str  = datetime.now().strftime("%Y%m%d_%H%M%S")
json_name =f"SkinCAP_{today_str}_open_end_QA.json"

dt=pd.read_csv(file_path)
SkinCAP_json = []
for index, data in dt.iterrows():
    disease = data['disease']
    caption_zh_polish_en = data['caption_zh_polish_en']
    skincap_file_path = data['skincap_file_path']
    caption_zh_polish = data['caption_zh_polish']
    caption_zh = data['caption_zh']
    if(disease.replace("-"," ") in caption_zh_polish_en):
        data_json={
            "image_name": skincap_file_path,
            "disease": disease,
            "caption_zh": caption_zh,
            "caption_zh_polish": caption_zh_polish,
            "answer":caption_zh_polish_en,
            "question_type":"open_end_QA"
            
        }
        SkinCAP_json.append(data_json)
        

output_path = Path(os.path.join(image_path,json_name))
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(SkinCAP_json, f, ensure_ascii=False, indent=4)

print(f"JSON save: {output_path.resolve()}")    

# close_end_QA

In [ ]:
import json
import os
import pandas as pd 
from pathlib import Path
from datetime import datetime 

image_path=r"/home/william/dataset/skin/SkinCAP"

file_path=r"/home/william/dataset/skin/SkinCAP/skincap_v240623.csv"
today_str  = datetime.now().strftime("%Y%m%d_%H%M%S")
json_name =f"SkinCAP_{today_str}_close_end_QA.json"

dt=pd.read_csv(file_path)
SkinCAP_json = []
for index, data in dt.iterrows():
    disease = data['disease']
    caption_zh_polish_en = data['caption_zh_polish_en']
    skincap_file_path = data['skincap_file_path']
    caption_zh_polish = data['caption_zh_polish']
    caption_zh = data['caption_zh']
    if(disease.replace("-"," ") in caption_zh_polish_en):
        data_json={
            "image_name": skincap_file_path,
            "caption_zh": caption_zh,
            "caption_zh_polish": caption_zh_polish,
            "caption_zh_polish_en":caption_zh_polish_en,
            "answer":disease.replace("-"," "),
            "question_type":"close_end_QA"
            
        }
        SkinCAP_json.append(data_json)
        

output_path = Path(os.path.join(image_path,json_name))
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(SkinCAP_json, f, ensure_ascii=False, indent=4)

print(f"JSON save: {output_path.resolve()}")    

# Multiple-Choice QA

In [ ]:
import json
import os
import pandas as pd
from pathlib import Path
from datetime import datetime
import random

# Path config
image_path = "/home/william/dataset/skin/SkinCAP"
csv_path = os.path.join(image_path, "skincap_v240623.csv")
today_str = datetime.now().strftime("%Y%m%d_%H%M%S")
json_name = f"SkinCAP_{today_str}_multiple_choice_QA.json"
output_path = Path(os.path.join(image_path, json_name))

# Load CSV
dt = pd.read_csv(csv_path)

# Collect unique disease labels
if "disease" not in dt.columns:
    raise ValueError("Column 'disease' is missing from the CSV file.")

all_labels = sorted(set(dt['disease'].dropna()))

# Early exit if too few labels
if len(all_labels) < 4:
    raise ValueError("Not enough distinct disease labels to construct 4-option questions.")

SkinCAP_json = []
for index, row in dt.iterrows():
    disease = str(row.get('disease', '')).strip()
    caption_zh = str(row.get('caption_zh', '')).strip()
    caption_zh_polish = str(row.get('caption_zh_polish', '')).strip()
    caption_zh_polish_en = str(row.get('caption_zh_polish_en', '')).strip()
    image_name = str(row.get('skincap_file_path', '')).strip()

    # Normalize for match
    norm_disease = disease.lower().replace("-", " ")
    norm_caption = caption_zh_polish_en.lower()

    if not disease or norm_disease not in norm_caption:
        continue

    # Sample 3 distractors
    try:
        distractors = random.sample([l for l in all_labels if l != disease], k=3)
    except ValueError:
        # If not enough distractors, skip this item
        continue

    # Construct options
    choices = [disease] + distractors
    random.shuffle(choices)
    option_strs = [f"{chr(65+i)}:{c}" for i, c in enumerate(choices)]
    question_str = ". ".join(option_strs) + "."

    # Build JSON item
    item = {
        "image_name": image_name,
        "caption_zh": caption_zh,
        "caption_zh_polish": caption_zh_polish,
        "caption_zh_polish_en": caption_zh_polish_en,
        "answer": disease.replace("-", " "),
        "question": question_str,
        "question_type": "multiple_choice_QA"
    }
    SkinCAP_json.append(item)

# Save JSON
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(SkinCAP_json, f, ensure_ascii=False, indent=4)

print(f"Saved {len(SkinCAP_json)} entries to: {output_path.resolve()}")


# Split skinCAP dataset

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

today_str = datetime.now().strftime("%Y%m%d_%H%M%S")
# Load the dataset
df = pd.read_csv("/home/william/dataset/skin/SkinCAP/skincap_v240623.csv")

# Remove rows with uninformative diagnosis captions
df = df[~df["caption_zh_polish_en"].str.contains(
    "This image does not provide enough information for an accurate diagnosis.", na=False
)]
df = df[~df["caption_zh"].str.contains(
    "这张图片无法准确诊断。", na=False
)]
# Keep only diseases with at least 2 samples
disease_counts = df['disease'].value_counts()
valid_diseases = disease_counts[disease_counts > 1].index
df_filtered = df[df['disease'].isin(valid_diseases)].copy()

# Initialize lists to hold train and test splits
train_list = []
test_list = []

# For each disease, split roughly 50% to train and 50% to test
for disease, group in df_filtered.groupby('disease'):
    n = len(group)
    n_train = n // 2  # Allocate half  to training
    train_grp = group.sample(n=n_train, random_state=42)
    test_grp = group.drop(train_grp.index)
    train_list.append(train_grp)
    test_list.append(test_grp)

# Combine and shuffle the final train and test sets
train_df = pd.concat(train_list).sample(frac=1, random_state=42).reset_index(drop=True)
test_df = pd.concat(test_list).sample(frac=1, random_state=42).reset_index(drop=True)
train_df["datatype"] = "train"
test_df["datatype"] = "test"

# save the splits to CSV
train_df.to_csv(f"/home/william/dataset/skin/SkinCAP/split/skincap_train_{today_str}.csv", index=False, encoding="utf-8-sig")
test_df.to_csv(f"/home/william/dataset/skin/SkinCAP/split/skincap_test_{today_str}.csv", index=False, encoding="utf-8-sig")


combined_df = pd.concat([train_df, test_df], ignore_index=True)
combined_df.to_csv(f"/home/william/dataset/skin/SkinCAP/split/skincap_all_{today_str}.csv", index=False, encoding="utf-8-sig")



train_counts = train_df["disease"].value_counts().head(30)
test_counts = test_df["disease"].value_counts().head(30)



plt.figure(figsize=(14, 8))
train_counts.plot(kind="bar", color="skyblue")
plt.title("Top 30 Diseases in Training Set")
plt.xlabel("Disease")
plt.ylabel("Count")
plt.xticks(rotation=45, ha="right")
plt.grid(axis='y')
plt.tight_layout()
plt.show()

plt.figure(figsize=(14, 8))
test_counts.plot(kind="bar", color="salmon")
plt.title("Top 30 Diseases in Test Set")
plt.xlabel("Disease")
plt.ylabel("Count")
plt.xticks(rotation=45, ha="right")
plt.grid(axis='y')
plt.tight_layout()
plt.show()

# Print dataset statistics
print(f"Number of training samples: {len(train_df)}\n")
print(f"training samples: {train_counts}")
print(f"Number of testing samples: {len(test_df)}\n")
print(f"Number of testing samples: {train_counts}")
print(f"Total number of samples: {len(train_df) + len(test_df)}")


# skinCAP test dataset

## open_end_QA

In [ ]:
import json
import os
import pandas as pd 
from pathlib import Path
from datetime import datetime 

image_path=r"/home/william/dataset/skin/SkinCAP/split"

file_path=r"/home/william/dataset/skin/SkinCAP/split/skincap_test_20250723_200925.csv"
today_str  = datetime.now().strftime("%Y%m%d_%H%M%S")
json_name =f"SkinCAP_{today_str}_open_end_QA.json"

dt=pd.read_csv(file_path)
SkinCAP_json = []
for index, data in dt.iterrows():
    disease = data['disease']
    caption_zh_polish_en = data['caption_zh_polish_en']
    skincap_file_path = data['skincap_file_path']
    caption_zh_polish = data['caption_zh_polish']
    caption_zh = data['caption_zh']
    # if(disease.replace("-"," ") in caption_zh_polish_en):
    #     data_json={
    #         "image_name": skincap_file_path,
    #         "disease": disease,
    #         "caption_zh": caption_zh,
    #         "caption_zh_polish": caption_zh_polish,
    #         "answer":caption_zh_polish_en,
    #         "question_type":"open_end_QA"
    #         
    #     }
    # SkinCAP_json.append(data_json)
    data_json={
        "image_name": skincap_file_path,
        "disease": disease,
        "caption_zh": caption_zh,
        "caption_zh_polish": caption_zh_polish,
        "answer":caption_zh_polish_en,
        "question_type":"open_end_QA"

    }
    SkinCAP_json.append(data_json)    

output_path = Path(os.path.join(image_path,json_name))
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(SkinCAP_json, f, ensure_ascii=False, indent=4)

print(f"Saved {len(SkinCAP_json)} entries to: {output_path.resolve()}")  

## close_end_QA

In [ ]:
import json
import os
import pandas as pd 
from pathlib import Path
from datetime import datetime 

image_path=r"/home/william/dataset/skin/SkinCAP/split"

file_path=r"/home/william/dataset/skin/SkinCAP/split/skincap_test_20250723_200925.csv"
today_str  = datetime.now().strftime("%Y%m%d_%H%M%S")
json_name =f"SkinCAP_{today_str}_close_end_QA.json"

dt=pd.read_csv(file_path)
SkinCAP_json = []
for index, data in dt.iterrows():
    disease = data['disease']
    caption_zh_polish_en = data['caption_zh_polish_en']
    skincap_file_path = data['skincap_file_path']
    caption_zh_polish = data['caption_zh_polish']
    caption_zh = data['caption_zh']
    # if(disease.replace("-"," ") in caption_zh_polish_en):
    data_json={
        "image_name": skincap_file_path,
        "caption_zh": caption_zh,
        "caption_zh_polish": caption_zh_polish,
        "caption_zh_polish_en":caption_zh_polish_en,
        "answer":disease.replace("-"," "),
        "question_type":"close_end_QA"
            
    }
    SkinCAP_json.append(data_json)
        

output_path = Path(os.path.join(image_path,json_name))
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(SkinCAP_json, f, ensure_ascii=False, indent=4)

print(f"Saved {len(SkinCAP_json)} entries to: {output_path.resolve()}")  

## multiple_choice_QA

In [ ]:
import json
import os
import pandas as pd
from pathlib import Path
from datetime import datetime
import random

# Path config
image_path=r"/home/william/dataset/skin/SkinCAP/split"
csv_path=r"/home/william/dataset/skin/SkinCAP/split/skincap_test_20250723_200925.csv"
today_str = datetime.now().strftime("%Y%m%d_%H%M%S")
json_name = f"SkinCAP_{today_str}_multiple_choice_QA.json"
output_path = Path(os.path.join(image_path, json_name))

# Load CSV
dt = pd.read_csv(csv_path)

# Collect unique disease labels
if "disease" not in dt.columns:
    raise ValueError("Column 'disease' is missing from the CSV file.")

all_labels = sorted(set(dt['disease'].dropna()))

# Early exit if too few labels
if len(all_labels) < 4:
    raise ValueError("Not enough distinct disease labels to construct 4-option questions.")

SkinCAP_json = []
for index, row in dt.iterrows():
    disease = str(row.get('disease', '')).strip()
    caption_zh = str(row.get('caption_zh', '')).strip()
    caption_zh_polish = str(row.get('caption_zh_polish', '')).strip()
    caption_zh_polish_en = str(row.get('caption_zh_polish_en', '')).strip()
    image_name = str(row.get('skincap_file_path', '')).strip()

    # Normalize for match
    norm_disease = disease.lower().replace("-", " ")
    norm_caption = caption_zh_polish_en.lower()

    # if not disease or norm_disease not in norm_caption:
    #     continue

    # Sample 3 distractors
    try:
        distractors = random.sample([l for l in all_labels if l != disease], k=3)
    except ValueError:
        # If not enough distractors, skip this item
        continue

    # Construct options
    choices = [disease] + distractors
    random.shuffle(choices)
    option_strs = [f"{chr(65+i)}:{c}" for i, c in enumerate(choices)]
    question_str = ". ".join(option_strs) + "."

    # Build JSON item
    item = {
        "image_name": image_name,
        "caption_zh": caption_zh,
        "caption_zh_polish": caption_zh_polish,
        "caption_zh_polish_en": caption_zh_polish_en,
        "answer": disease.replace("-", " "),
        "question": question_str,
        "question_type": "multiple_choice_QA"
    }
    SkinCAP_json.append(item)

# Save JSON
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(SkinCAP_json, f, ensure_ascii=False, indent=4)

print(f"Saved {len(SkinCAP_json)} entries to: {output_path.resolve()}")


# SkinCAP_train image

In [ ]:
import os
import shutil
import pandas as pd


csv_path = "/home/william/dataset/skin/SkinCAP/split/skincap_train_20250723_200925.csv"
src_image_dir = "/home/william/dataset/skin/SkinCAP/skincap"
dst_image_dir = "/home/william/dataset/skin/SkinCAP/SkinCAP_train"


os.makedirs(dst_image_dir, exist_ok=True)


df = pd.read_csv(csv_path)


for image_name in df["skincap_file_path"]:
    src_path = os.path.join(src_image_dir, image_name)
    dst_path = os.path.join(dst_image_dir, image_name)
    if os.path.exists(src_path):
        shutil.copy2(src_path, dst_path)
    else:
        print(f"Warning: File not found - {src_path}")

print("Image copy completed.")


# copy skinCAP image

In [ ]:
import os
import json
import shutil


json_path = "/home/william/dataset/skin/SkinCAP/split/SkinCAP_20250723_202509_close_end_QA.json"
source_dir = "/home/william/dataset/skin/SkinCAP/skincap"
target_dir = "/home/william/dataset/skin/SkinCAP/skincap_test_10"


os.makedirs(target_dir, exist_ok=True)


with open(json_path, "r", encoding="utf-8") as f:
    data = json.load(f)


for item in data[0:10]:
    image_name = item.get("image_name")
    if image_name:
        src_path = os.path.join(source_dir, image_name)
        dst_path = os.path.join(target_dir, image_name)
        if os.path.exists(src_path):
            shutil.copy2(src_path, dst_path)
        else:
            print(f"File not found: {src_path}")

print("Copy completed.")


#  train close_end_QA

In [ ]:
import json
import os
import pandas as pd 
from pathlib import Path
from datetime import datetime 

image_path=r"/home/william/model/LLaMA-Factory/data"

file_path=r"/home/william/dataset/skin/SkinCAP/split/skincap_train_20250723_200925.csv"
today_str  = datetime.now().strftime("%Y%m%d_%H%M%S")
# json_name =f"train_SkinCAP_{today_str}_close_end_QA.json"
json_name =f"train_SkinCAP_close_end_QA.json"

dt=pd.read_csv(file_path)
SkinCAP_json = []
for index, data in dt.iterrows():
    disease = data['disease']
    caption_zh_polish_en = data['caption_zh_polish_en']
    skincap_file_path = data['skincap_file_path']
    caption_zh_polish = data['caption_zh_polish']
    caption_zh = data['caption_zh']
    # if(disease.replace("-"," ") in caption_zh_polish_en):
    data_json={
        "instruction":"<image> What is the name of the disease shown in the image?\n Answer the question using a single word or phrase.",
        "input": "",
        "output": disease.replace("-"," "),
        "images": ["train_SkinCAP/"+skincap_file_path]   
    }
    SkinCAP_json.append(data_json)
        

output_path = Path(os.path.join(image_path,json_name))
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(SkinCAP_json, f, ensure_ascii=False, indent=4)

print(f"Saved {len(SkinCAP_json)} entries to: {output_path.resolve()}")  

In [ ]:
import json
import os
import pandas as pd 
from pathlib import Path
from datetime import datetime 

image_path=r"/home/william/model/LLaMA-Factory/data"

file_path=r"/home/william/dataset/skin/SkinCAP/split/skincap_test_20250723_200925.csv"
today_str  = datetime.now().strftime("%Y%m%d_%H%M%S")
# json_name =f"train_SkinCAP_{today_str}_close_end_QA.json"
json_name =f"eval_SkinCAP_close_end_QA.json"

dt=pd.read_csv(file_path)
SkinCAP_json = []
for index, data in dt.iterrows():
    disease = data['disease']
    caption_zh_polish_en = data['caption_zh_polish_en']
    skincap_file_path = data['skincap_file_path']
    caption_zh_polish = data['caption_zh_polish']
    caption_zh = data['caption_zh']
    # if(disease.replace("-"," ") in caption_zh_polish_en):
    data_json={
        "instruction":"<image> What is the name of the disease shown in the image?\n Answer the question using a single word or phrase.",
        "input": "",
        "output": disease.replace("-"," "),
        "images": ["eval_SkinCAP/"+skincap_file_path]   
    }
    SkinCAP_json.append(data_json)
        

output_path = Path(os.path.join(image_path,json_name))
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(SkinCAP_json, f, ensure_ascii=False, indent=4)

print(f"Saved {len(SkinCAP_json)} entries to: {output_path.resolve()}")  